In [0]:
# Define os parâmetros da Job
dbutils.widgets.text('p_path','')
dbutils.widgets.text('p_dataset_name','')
dbutils.widgets.text('p_path_layer_name','')

### Lê arquivo kaggle.json com as credenciais de acesso à API e cria variavies de conexão

In [0]:
dbutils.fs.cp("dbfs:/FileStore/kaggle.json", "file:/root/.kaggle/kaggle.json")

True

In [0]:
# Importa bibliotecas
from kaggle.api.kaggle_api_extended import KaggleApi    # Lib já instalada no cluster
from shutil import rmtree
import os
import json
import tempfile

In [0]:
def read_kaggle_credentials(file_path_json: str) -> tuple:
    """
    Lê as credenciais do Kaggle de um arquivo JSON armazenado no caminho especificado.

    Parâmetros:
    file_path (str): Caminho completo até o arquivo JSON.

    Retorna:
    tuple: Contém o nome de usuário e a chave do Kaggle, e um booleano indicando se ambas as credenciais são não nulas.
    """

    try:
        # Lê o arquivo JSON
        with open(file_path_json, 'r') as file:
            credentials = json.load(file)
        
        # Extrai as credenciais
        KAGGLE_USERNAME = credentials['username']
        KAGGLE_KEY = credentials['key']
        
        # Verifica se ambas as credenciais são não nulas
        credentials_found = bool(KAGGLE_USERNAME and KAGGLE_KEY)
        return (KAGGLE_USERNAME, KAGGLE_KEY, credentials_found)
    except Exception as e:
        print(f"Erro ao ler as credenciais: {e}")
        return (None, None, False)

In [0]:
def autentifica_kaggle(KAGGLE_USERNAME :str, KAGGLE_KEY :str) -> KaggleApi:
    """
    Autentica na API do Kaggle utilizando as credenciais fornecidas.

    Parâmetros:
    KAGGLE_USERNAME (str): Nome de usuário do Kaggle.
    KAGGLE_KEY (str): Chave de API do Kaggle.

    Retorna:
    KaggleApi: Instância da API do Kaggle autenticada.
    """

    # Define as variavéis de ambiente
    os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
    os.environ['KAGGLE_KEY'] = KAGGLE_KEY

    # Cria e autentica uma instância da API do Kaggle
    api = KaggleApi()
    api.authenticate()

    print('Conexão estabelecida com sucesso!')

    return api

In [0]:
def download_kaggle_dataset_adsl(dataset_name, path_adsl, KAGGLE_USERNAME: str, KAGGLE_KEY: str)->None:
    """
    Baixa e extrai um dataset do Kaggle e salva no Azure Data Lake Storage.

    Parâmetros:
    dataset_name (str): Nome do dataset no Kaggle.
    path_adsl (str): Caminho no Azure Data Lake Storage onde o dataset será salvo.
    KAGGLE_USERNAME (str): Nome de usuário do Kaggle.
    KAGGLE_KEY (str): Chave de API do Kaggle.
    """

    # Tentativa de Autentificação
    try:
        api = autentifica_kaggle(KAGGLE_USERNAME, KAGGLE_KEY)
    except Exception as e :
        print(f"Erro de autenticação: {e}")
        print('Verifique se as credenciais estão corretas e se a biblioteca kaggle está instalada')
        return  # Encerra a função se a autenticação falhar
    
    # Tentativa de download e extração do dataset
    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            api.dataset_download_files(dataset_name, path=tmpdir, unzip=True)                   # Tentativa de download e extração do dataset

            arquivos = os.listdir(tmpdir)
            for arquivo in arquivos:
                file_path = os.path.join(tmpdir, arquivo)
                path_destination_adsl = os.path.join(path_adsl, arquivo)
                dbutils.fs.cp(f"file:{file_path}", path_destination_adsl)
                print(f"* Arquivo {arquivo} salvo em {path_destination_adsl}")
    except Exception as e:
        print(f"Erro ao baixar o dataset: {e}")
        return

In [0]:
def download_kaggle_dataset_dbfs(dataset_name, path_dbfs, KAGGLE_USERNAME: str, KAGGLE_KEY: str)->None:
    """
    Baixa e extrai um dataset do Kaggle e salva no Azure Data Lake Storage.

    Parâmetros:
    dataset_name (str): Nome do dataset no Kaggle.
    path_adsl (str): Caminho no Azure Data Lake Storage onde o dataset será salvo.
    KAGGLE_USERNAME (str): Nome de usuário do Kaggle.
    KAGGLE_KEY (str): Chave de API do Kaggle.
    """
    
    # Tentativa de Autentificação
    try:
        api = autentifica_kaggle(KAGGLE_USERNAME, KAGGLE_KEY)
    except Exception as e :
        print(f"Erro de autenticação: {e}")
        print('Verifique se as credenciais estão corretas e se a biblioteca kaggle está instalada')
        return  # Encerra a função se a autenticação falhar
    
    # Tentativa de download e extração do dataset
    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            api.dataset_download_files(dataset_name, path=tmpdir, unzip=True)                   # Tentativa de download e extração do dataset

            arquivos = os.listdir(tmpdir)
            for arquivo in arquivos:
                file_path = os.path.join(tmpdir, arquivo)
                path_destination_dbfs = os.path.join(path_dbfs, arquivo)
                dbutils.fs.cp(f"file:{file_path}", path_destination_dbfs)
                print(f"* Arquivo {arquivo} salvo em {path_destination_dbfs}")
    except Exception as e:
        print(f"Erro ao baixar o dataset: {e}")
        return

In [0]:
# Função principal que orquestra a execução das funções acima
def main():
    # Define os valores para os parâmetros (Valores passados na Job)

    # path = 'abfss://datum-metastore@proofconceptdatum.dfs.core.windows.net'
    # path_layer_name = 'raw'
    # file_name_json = 'kaggle.json'
    # dataset_name = 'marian447/retail-store-sales-transactions'

    path = dbutils.widgets.get('p_path')
    path_layer_name = dbutils.widgets.get('p_path_layer_name')
    dataset_name = dbutils.widgets.get('p_dataset_name')

    path_adsl = f'{path}/{path_layer_name}/'
    path_dbfs = f'{path}/{path_layer_name}/'
    file_path_json = '/root/.kaggle/kaggle.json'

    # Recupera as credencias executando a função read_kaggle_credentials
    credentials = read_kaggle_credentials(file_path_json=file_path_json)
    user = credentials[0]
    key = credentials[1]

    # Faz o download, extrai o dataset e salva na pasta especificada executando a função download_kaggle_dataset
    # download_kaggle_dataset_adsl(dataset_name=dataset_name, path_adsl=path_adsl, KAGGLE_USERNAME=user, KAGGLE_KEY=key)
    download_kaggle_dataset_dbfs(dataset_name=dataset_name, path_dbfs=path_dbfs, KAGGLE_USERNAME=user, KAGGLE_KEY=key)

In [0]:
# Bloco que verifica se o script é o ponto de entrada principal
if __name__ == "__main__":
    main()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-508162604964217>, line 3
      1 # Bloco que verifica se o script é o ponto de entrada principal
      2 if __name__ == "__main__":
----> 3     main()

NameError: name 'main' is not defined

#### _A função criada "download_kaggle_dataset_adsl" foi mais pra exemplificar como trabalhar com serviço em cloud. Devido a quantidade baixa de dados, vamos optar por trabalhar com os arquivos no DBFS_